In [181]:
def parenthese_match(string):
    """take from string until matching close parenthese found.
    string should start with the ( that is to be to matched."""
    
    open_parentheses = 0
    close_parentheses = 0
    
    index = 0
    count = 0
    
    for char in string:
        count += 1
        
        if char == "(":
            open_parentheses += 1
            
        elif char == ")":
            close_parentheses += 1
            
            if open_parentheses == close_parentheses:
                index += count
                break
                
    if index == 0:
        print("Parse error. Trailing close parentheses not found.")
        return
    
    else:
        return string[0:index]

In [182]:
parenthese_match("(CI(KI))(CIK)")

'(CI(KI))'

In [183]:
parenthese_match("(KI)(CIK)")

'(KI)'

In [184]:
parenthese_match("(KI(WI(BI)))(CIK)")

'(KI(WI(BI)))'

Example expression `S(KS)Kxyz` and example list `["S",["K","S"],"K","x","y","z"]`

Another example expression `S(CI(KI))(CIK)` and example list `["S",["C","I",["K","I"]],["C","I","K"]]`

In [185]:
def encoding(data):
    """CL string to list OR CL list to string"""
    
    if type(data) == str:
        d = data 
        _list = []
        
        while len(d) > 0:
            if d[0] == "(":
                p = parenthese_match(d) 
                _list.append(encoding(p[1:-1])) 
                d = d[len(p):]
                
            elif d[0] == ")":
                d = d[1:]
                                
            else:
                _list.append(d[0])
                d = d[1:]
        
        if len(_list) == 1 and type(_list[0]) == list:
            return _list[0]
        
        else:
            return _list
        
    elif type(data) == list:
        string = ""
        
        for i in data:
            if type(i) == str:
                string += i
            else:
                string += "("
                string += encoding(i)
                string += ")"
        
        return string 
        
    else:
        return f"Type Error. Got: {type(data)}, Expected 'str' or 'list'"

In [186]:
encoding(["S",["K","S"],"K","x","y","z"])

'S(KS)Kxyz'

In [187]:
encoding(["S",["C","I",["K","I"]],["C","I","K"]])

'S(CI(KI))(CIK)'

In [188]:
encoding((1,2))

"Type Error. Got: <class 'tuple'>, Expected 'str' or 'list'"

In [189]:
encoding("S(KS)Kxyz")

['S', ['K', 'S'], 'K', 'x', 'y', 'z']

In [190]:
encoding("S(KS)Kxyz") == ["S",["K","S"],"K","x","y","z"]

True

In [191]:
encoding("S(CI(KI))(CIK)")

['S', ['C', 'I', ['K', 'I']], ['C', 'I', 'K']]

In [192]:
encoding("S(CI(KI))(CIK)") == ["S",["C","I",["K","I"]],["C","I","K"]]

True

-- --

In [303]:
def I_combinator(string):
    """Ix = x"""
    return string

def K_combinator(string):
    """Kxy = x"""
    if type(encoding(string)[0]) == str:
        return encoding(string)[0]
    else:
        return encoding(encoding(string)[0])

def B_combinator(string):
    """Bxyz = x(yz)"""
    l = encoding(string)
    return encoding([l[0],[l[1],l[2]]])

def C_combinator(string):
    """Cxyz = xzy"""
    l = encoding(string)
    return encoding([l[0],l[2],l[1]])
    
def W_combinator(string):
    """Wxy = xyy"""
    l = encoding(string)
    return encoding([l[0],l[1],l[1]])

def S_combinator(string):
    """Sxyz = xz(yz)"""
    l = encoding(string)
    return encoding([l[0],l[2],[l[1],l[2]]])

def Y_combinator(string):
    """Yx = B(WI)(BWB)x"""
    l = encoding(string)
    return B_combinator("(WI)(BWB)" + string)

def M_combinator(string):
    """Mx = xx"""
    return string + string

def F_combinator(string):
    """Fxy = y"""
    if type(encoding(string)[1]) == str:
        return encoding(string)[1]
    else:
        return encoding(encoding(string)[1])

In [219]:
I_combinator("x")

'x'

In [220]:
K_combinator("xy")

'x'

In [221]:
B_combinator("xyz")

'x(yz)'

In [222]:
C_combinator("xyz")

'xzy'

In [223]:
W_combinator("xy")

'xyy'

In [224]:
S_combinator("xyz")

'xz(yz)'

In [225]:
Y_combinator("x")

'(WI)((BWB)x)'

In [226]:
M_combinator("x")

'xx'

In [304]:
F_combinator("xy")

'y'

-- --

In [201]:
encoding("(ABC)")

['A', 'B', 'C']

In [202]:
m = {"S": S_combinator}

In [203]:
m["S"]("xyz")

'xz(yz)'

In [227]:
def evaluate(string):
    cl_list = encoding(string)
    
    while type(cl_list[0]) == list: # strip leading parentheses
        cl_list = [i for i in cl_list[0]] + cl_list[1:]
    
    # "X": (function, arg_num)
    _map = {"I": (I_combinator,1), "K": (K_combinator,2), "B": (B_combinator,3), "C": (C_combinator,3), 
            "W": (W_combinator,2), "S": (S_combinator,3), "Y": (Y_combinator,1), "M": (M_combinator,1)}
    
    combinator_return = _map[cl_list[0]][0](encoding(cl_list[1:(_map[cl_list[0]][1]+1)]))
    tail = encoding(cl_list[(_map[cl_list[0]][1]+1):])
    
    return combinator_return + tail

In [228]:
evaluate("S(KS)Kxyz")

'(KS)x(Kx)yz'

In [229]:
evaluate("(KS)x(Kx)yz")

'S(Kx)yz'

In [230]:
evaluate("S(Kx)yz")

'(Kx)z(yz)'

In [231]:
evaluate("(Kx)z(yz)")

'x(yz)'

-- --

In [279]:
def recur(string,print_steps=False,write_steps=False):
    steps = [string]

    while True:
        try:
            ns = evaluate(string)
            steps.append(ns)
            string = ns
                
            # cycle
            if len(set(steps)) != len(steps) : 
                break
            
            # no cycle
            else: 
                pass
            
        except:
            break
    
    if print_steps:
        for i in steps:
            print(i)
            
    if write_steps:
        file = input("filename : ")
        
        with open(file + ".txt", "w") as f:
            for i in steps:
                f.write(i)
                f.write("\n")
        
    return steps[-1]

In [280]:
recur("S(KS)Kxyz",print_steps=True)

S(KS)Kxyz
(KS)x(Kx)yz
S(Kx)yz
(Kx)z(yz)
x(yz)


'x(yz)'

In [281]:
recur("CI(KI)(BC(CI)xy)",print_steps=True)

CI(KI)(BC(CI)xy)
I(BC(CI)xy)(KI)
(BC(CI)xy)(KI)
C((CI)x)y(KI)
((CI)x)(KI)y
I(KI)xy
(KI)xy
Iy
y


'y'

In [282]:
recur("K(C(CI(KI))K)I(KI)(C(CI(KI))K)I(KI)(C(CI(KI))K)IK(C(CII)(C(CI(KI))K)K(KI))(KI)(KI)K",print_steps=True)

K(C(CI(KI))K)I(KI)(C(CI(KI))K)I(KI)(C(CI(KI))K)IK(C(CII)(C(CI(KI))K)K(KI))(KI)(KI)K
C(CI(KI))K(KI)(C(CI(KI))K)I(KI)(C(CI(KI))K)IK(C(CII)(C(CI(KI))K)K(KI))(KI)(KI)K
(CI(KI))(KI)K(C(CI(KI))K)I(KI)(C(CI(KI))K)IK(C(CII)(C(CI(KI))K)K(KI))(KI)(KI)K
I(KI)(KI)K(C(CI(KI))K)I(KI)(C(CI(KI))K)IK(C(CII)(C(CI(KI))K)K(KI))(KI)(KI)K
(KI)(KI)K(C(CI(KI))K)I(KI)(C(CI(KI))K)IK(C(CII)(C(CI(KI))K)K(KI))(KI)(KI)K
IK(C(CI(KI))K)I(KI)(C(CI(KI))K)IK(C(CII)(C(CI(KI))K)K(KI))(KI)(KI)K
K(C(CI(KI))K)I(KI)(C(CI(KI))K)IK(C(CII)(C(CI(KI))K)K(KI))(KI)(KI)K
C(CI(KI))K(KI)(C(CI(KI))K)IK(C(CII)(C(CI(KI))K)K(KI))(KI)(KI)K
(CI(KI))(KI)K(C(CI(KI))K)IK(C(CII)(C(CI(KI))K)K(KI))(KI)(KI)K
I(KI)(KI)K(C(CI(KI))K)IK(C(CII)(C(CI(KI))K)K(KI))(KI)(KI)K
(KI)(KI)K(C(CI(KI))K)IK(C(CII)(C(CI(KI))K)K(KI))(KI)(KI)K
IK(C(CI(KI))K)IK(C(CII)(C(CI(KI))K)K(KI))(KI)(KI)K
K(C(CI(KI))K)IK(C(CII)(C(CI(KI))K)K(KI))(KI)(KI)K
C(CI(KI))KK(C(CII)(C(CI(KI))K)K(KI))(KI)(KI)K
(CI(KI))KK(C(CII)(C(CI(KI))K)K(KI))(KI)(KI)K
IK(KI)K(C(CII)(C(CI(KI))K)K(KI))(KI)(

'K'

In [283]:
recur("Yx")

'x(((BWB)x)((BWB)x))'

In [284]:
recur("WWW")

'WWW'

-- --

In [285]:
evaluate("BC(CI)xy")

'C((CI)x)y'

In [286]:
recur("CIK(C((CI)x)y)")

'x'

In [287]:
recur("CI(KI)(C((CI)x)y)")

'y'

In [288]:
recur("CI(KI)(C((CI)x)y)",write_steps=True)

filename : test


'y'

-- --

In [289]:
recur("BC(CI)(KI)(KI)(C(C(B(BK))z)(C(C(B(BK))y)(C(C(B(BK))x)(KI))))")

'(C(C(B(BK))y)(C(C(B(BK))x)(KI)))'

In [300]:
recur("C(C(B(BK))x)(C(C(B(BK))y)(C(C(B(BK))z)(KI)))")

'C(C(B(BK))x)(C(C(B(BK))y)(C(C(B(BK))z)(KI)))'

In [302]:
recur("B(WI)(BWB)(B(B(C(B(B(B(C(BC)I)(BC(CI)))(BC)))(BC(C(B(BK)))))B)B)(C(C(B(BK))x)(C(C(B(BK))y)(C(C(B(BK))z)(KI))))",write_steps=True)

filename : test


'C(((BC(CI))((BC)((B(B(((BWB)(B(B(C(B(B(B(C(BC)I)(BC(CI)))(BC)))(BC(C(B(BK)))))B)B))((BWB)(B(B(C(B(B(B(C(BC)I)(BC(CI)))(BC)))(BC(C(B(BK)))))B)B)))))(BC(C(B(BK)))))))I)(C(C(B(BK))x)(C(C(B(BK))y)(C(C(B(BK))z)(KI))))'

-- --

In [306]:
recur("C(B(B(C(B(C(B(B(CBC)(B(C(B(C(B(C(B(B(B(C(B(B(CB(CC(KI)))B))(KI))C))C)K))C))(K(KI))))))I))C)))KK(KI)KK(KI)",write_steps=True)

filename : test


''